# ML model deployment

Deploy the trained ML model into an Amazon SageMaker Model and save the predictions for the test dataframe.

In [ ]:
!pip install jsonlines

In [ ]:
import os
import boto3
import botocore
import numpy as np
import pandas as pd
import json
import jsonlines
import sagemaker
from sagemaker.predictor import json_serializer, json_deserializer
from sagemaker.amazon.amazon_estimator import get_image_uri


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt

#### Functions

In [ ]:
def download_object(bucket_name, key, local_path):
    """Download S3 object to local"""
    s3 = boto3.resource('s3')
    try:
        s3.Bucket(bucket_name).download_file(key,local_path)
    except botocore.exceptions.ClientError as e:
        if e.response['Error']['Code'] == "404":
            print("The object does not exist")
        else:
            raise

In [ ]:
def create_dir(directory):
    """Create a directory"""
    if not os.path.exists(directory):
        os.makedirs(directory)

In [ ]:
bucket = "YOUR_BUCKET_HERE"
prefix_in = "connect/O2VInput"
prefix_out = "connect/O2VOutput"
s3_client = boto3.client('s3')

In [ ]:
create_dir('./meta/')
create_dir('./data/')

download_object(bucket, os.path.join(prefix_in,'test','test.jsonl'), './data/test.jsonl')
download_object(bucket, os.path.join(prefix_in,'meta','vocab_to_token_dict.p'), './meta/vocab_to_token_dict.p')

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()

##### Specify the job_name

Specify the job name from the ML model that you would like to evaluate and deploy.

In [ ]:
job_name = "default2021-05-31-14-57-38" 

##### Create an Amazon SageMaker Model and deploy it to an endpoint.

Get the Amazon SageMaker model trained and create an endpoint to host it.

In [ ]:
model_data = f's3://{bucket}/{prefix_out}/{job_name}/output/model.tar.gz'
container = get_image_uri(boto3.Session().region_name, 'object2vec')

In [ ]:
trainedmodel = sagemaker.model.Model(
    model_data= model_data,
    image_uri= container,
    role=role,
    name=job_name) 

In [ ]:
trainedmodel.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

# Custom serializer 

In [ ]:
from sagemaker.serializers import SimpleBaseSerializer, JSONSerializer
import pickle
from nltk import word_tokenize

class O2VTextSerializer(SimpleBaseSerializer):
    def load_vocab_to_tokens(self, file_name):
        self.vocab_to_tokens = pickle.load(open(file_name,'rb'))

    def set_tokenizer(self, tokenizer):
        self.tokenizer = tokenizer

    def sentence_to_tokens(self,sentence):
        """converts sentences to tokens"""
        words = self.tokenizer(sentence)
        return [ self.vocab_to_tokens[w] for w in words if w in self.vocab_to_tokens]
    
    def serialize(self, data):
        js = {'instances': []}
        for row in data['instances']:
            print(row)
            new_row = row
            if type(new_row['in0'])==str:
                new_row['in0'] = self.sentence_to_tokens(row['in0'])
            if type(new_row['in1'])==str:
                new_row['in0'] = self.sentence_to_tokens(row['in0'])
            
            print(new_row)
            js['instances'].append(new_row)
            
        return json.dumps(js)


##### 4. Define predictor

Use Amazon SageMaker endpoint to retrieve the predictions of our test dataset (test.jsonl)

In [ ]:
serializer = O2VTextSerializer(content_type='application/json')
serializer.load_vocab_to_tokens('./meta/vocab_to_token_dict.p')
serializer.set_tokenizer(word_tokenize)

predictor = sagemaker.predictor.Predictor(
    endpoint_name=trainedmodel.endpoint_name,
    serializer=serializer,
    deserializer=sagemaker.deserializers.JSONDeserializer())

In [ ]:
test_payload = { 'instances':
                [
                    {
                        'in0': "Looks like it's working.",
                        'in1': [0]
                    }
                ]
               }

In [ ]:
predictor.predict(test_payload)

In [ ]:
sess = sagemaker.Session()
sess.delete_endpoint(trainedmodel.endpoint_name)

In [ ]:
sess.delete_endpoint_config(trainedmodel.endpoint_name )